# Dense neural network with PyTorch
Authors: Javier Duarte, Tyler Mitchell 

In [ ]:
import sys
!{sys.executable} -m pip install torch torchvision root_pandas --user

In [ ]:
import torch
import numpy as np
import root_pandas
import pandas as pd

In [ ]:
# Copy TTree HZZ4LeptonsAnalysisReduced into a pandas DataFrame.
filename = {
    'VV': root_pandas.read_root('data/ntuple_4mu_VV.root', key='HZZ4LeptonsAnalysisReduced'),
    'bkg': root_pandas.read_root('data/ntuple_4mu_bkg.root', key='HZZ4LeptonsAnalysisReduced')
}

# Drop all variables except for those we want to use when training.
VARS = ['f_mass4l','f_massjj']
df = {
    'VV': filename['VV'][VARS],
    'bkg': filename['bkg'][VARS],
}

In [ ]:
# Make sure the inputs are well behaved.
df['VV']= df['VV'][(df['VV'][VARS[0]] > -999) & (df['VV'][VARS[1]] > -999)]
df['bkg']= df['bkg'][(df['bkg'][VARS[0]] > -999) & (df['bkg'][VARS[1]] > -999)] 

In [ ]:
# add isSignal variable
df['VV']['isSignal'] = np.ones(len(df['VV'])) 
df['bkg']['isSignal'] = np.zeros(len(df['bkg'])) 

In [ ]:
# Combine signal and background into one DataFrame then split into input variables and labels.
NDIM = len(VARS)
df_all = pd.concat([df['VV'],df['bkg']])
dataset = df_all.values
X = dataset[:,0:NDIM]
Y = dataset[:,NDIM]

# Split into training and testing data.
from sklearn.model_selection import train_test_split
X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
print(X_train_val)
print(X)

# preprocessing: standard scalar (reshape inputs to mean=0, variance=1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train_val)
X_train_val = scaler.transform(X_train_val)
X_test = scaler.transform(X_test)

# Split again, this time into training and validation data.
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=0.2, random_state=7)

In [ ]:
# Build our model. 
import torch
model = torch.nn.Sequential(
    torch.nn.Linear(2, 20),
    torch.nn.Sigmoid(),
    torch.nn.Linear(20, 20),
    torch.nn.Sigmoid(),
    torch.nn.Linear(20, 1),
    torch.nn.Sigmoid()
)

# Use Binary Cross Entropy as our loss function.
loss_fn = torch.nn.BCELoss()

# Optimize the model parameters using the Adam optimizer.
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
import math
def getBatches(N, tot):
    """Function to find nearest acceptable batch size"""
    print N, tot
    if tot % N == 0:
        return N
    closest = 1
    for i in xrange(1, N):
        if tot % i == 0 and abs(N - i) < abs(N-closest):
            closest = i
    return closest
    
# Create batches from total data. We have to make sure the batch size is an appropriate divisor of the total 
# number of training events
N = getBatches(1024, len(X_train))
unbatched_x = torch.from_numpy(X_train).float()
x = unbatched_x.view(-1, N, len(VARS))
unbatched_y = torch.from_numpy(Y_train).float()
y_b = unbatched_y.view(-1, N, 1)

In [ ]:
# Get validation data ready
val_data = torch.from_numpy(X_val).float()
val_label = torch.from_numpy(Y_val).float()

In [ ]:
losses, val_losses = [], []
min_loss, stale_epochs = 100., 0

# 500 epochs. 
for t in xrange(500):
    batch_loss, val_batch_loss = [], []
    for b in xrange(len(x)):
        
        # Forward pass: make a prediction for each x event in batch b.
        y_pred = model(x[b])

        # Get the labels.
        label = y_b[b]
        y = label.view_as(y_pred)  # reshape label data to the shape of y_pred

        # Compute and print loss.
        loss = loss_fn(y_pred, y)
        batch_loss.append(loss.item())

        # Let's look at the validation set.
        
        # Torch keeps track of each operation performed on a Tensor, so that it can take the gradient later.
        # We don't need to store this information when looking at validation data, so turn it off with
        # torch.no_grad().
        with torch.no_grad():
            
            # Forward pass on validation set.
            output = model(val_data)

            # Get labels and compute loss again
            val_y = val_label.view_as(output)
            val_loss = loss_fn(output, val_y)
            val_batch_loss.append(val_loss.item())

            # Monitor the loss function to prevent overtraining.
            if stale_epochs > 20:
                break

            if val_loss.item() - min_loss < 0:
                min_loss = val_loss.item()
                stale_epochs = 0
            else:
                stale_epochs += 1

        #print(t, b, loss.item(), val_loss.item())

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
        
    losses.append(min(batch_loss))
    val_losses.append(min(val_batch_loss))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

with torch.no_grad():
    # plot loss vs epoch
    plt.figure(figsize=(16,8))
    ax = plt.subplot(1, 2, 1)
    ax.plot(losses, label='loss')
    ax.plot(val_losses, label='val_loss')
    ax.legend(loc="upper right")
    plt.show()


    # Plot ROC
    X_test_in = torch.from_numpy(X_test).float()
    Y_predict = model(X_test_in)
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, thresholds = roc_curve(Y_test, Y_predict)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(16,8))
    ax2 = plt.subplot(1, 2, 2)
    ax2.plot(fpr, tpr, lw=2, color='cyan', label='auc = %.3f' % (roc_auc))
    ax2.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k', label='random chance')
    ax2.set_xlim([0, 1.0])
    ax2.set_ylim([0, 1.0])
    ax2.set_xlabel('false positive rate')
    ax2.set_ylabel('true positive rate')
    ax2.set_title('receiver operating curve')
    ax2.legend(loc="lower right")
    plt.show()

In [ ]:
# Hint: We want to make a three 2D numpy arrays: 
# x values at each (x, y) grid point
# y values at each (x, y) grid point
# z values (model prediction) at each (x, y) grid point

myXI, myYI = np.meshgrid(np.linspace(-2, 2, 200), np.linspace(-2, 2, 200))
# print shape
print(myXI.shape)

myZI = np.zeros(myXI.shape)

for i in range(0, len(myXI)):
    for j in range(0, len(myYI)):
        myXI[i,j] # x value of xi, yj point
        myYI[i,j] # y value of xi, yj point
        data_point = torch.tensor([myXI[i,j], myYI[i,j]])
        myZI[i,j] = model(data_point)

In [ ]:
print(myZI)
myZI = model(torch.from_numpy(np.c_[myXI.ravel(), myYI.ravel()]).float())
myZI = myZI.reshape(myXI.shape)

In [ ]:
from matplotlib.colors import ListedColormap
plt.figure(figsize=(20,7))

# plot contour map of NN output
# overlaid with test data points
ax = plt.subplot(1, 2, 1)
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

cont_plot = ax.contourf(myXI, myYI, myZI>0.5, cmap=cm, alpha=.8)
ax.scatter(X_test[:, 0], X_test[:, 1], c=Y_test, cmap=cm_bright, edgecolors='k')
ax.set_xlim(-2,2)
ax.set_ylim(-2,2)
ax.set_xlabel(VARS[0])
ax.set_ylabel(VARS[1])
plt.colorbar(cont_plot,ax=ax, boundaries=[0,1],label='NN output')

# plot decision boundary
# overlaid with test data points